In [43]:
# snapnums_cptmarvel = [
#     '004096' 
#     # '003968', '003840', '003712', '003636', '003584', '003456', '003328', '003245', '003200', 
#     #                   '003072', '002944', '002816', '002688', '002624', '002560', '002432', '002304', '002176', '002162',
#     #                   '002048', '001920', '001813', '001792', '001664', '001543', '001536', '001408', '001331', '001280', 
#     #                   '001162', '001152', '000896', '000818', '000768', '000672', '000640', '000512', '000482', '000291',
#     #                   '000199', '000146', 
#     #                   '000128'
#                       ]

# filepaths_cptmarvel = [SIM_FOLDER_PATH+'cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/snapshots_200crit_cptmarvel/cptmarvel.cosmo25cmb.4096g5HbwK1BH.'+s for s in snapnums_cptmarvel]

# # properties to check
# properties = {
#     "s.properties": [
#         "time"
#     ],
#     "s.gas": [
#         "x", "y", "z",
#         "iord",
#         "vx",
#         "vy",
#         "vz"
#     ],
#     "s.gas.g": [
#         "rho",
#         "temp",
#         "mass",
#         "coolontime"
#     ],
#     "h.properties": [
#         "Rvir",
#         "Xc", "Yc", "Zc",
#         "VXc", "VYc", "VZc",
#         "Xc", "Yc", "Zc",
#         "VXc", "VYc", "VZc",
#         "M_star",
#         "M_gas",
#         "M_star",
#         "M_gas"
#     ]
# }

# df = pd.DataFrame()
# for fp in filepaths_cptmarvel:
#     s = pynbody.load(fp)
#     h = s.halos(halo_numbers='v1')
#     halo = h[1]
#     curout = pd.DataFrame()
#     spropkeys = s.properties.keys()
#     sgaskeys = s.gas.keys()
#     sgasgkeys = s.gas.g.keys()  
#     hpropkeys = halo.properties.keys()
#     curout["snapnum"] = [fp[-4:]]
#     print(f'Checking properties for snapshot {fp[-4:]}')
#     for prop in properties['s.properties']:
#         curout[prop] = [prop in spropkeys]
#     for prop in properties['s.gas']:
#         curout[prop] = [prop in sgaskeys]
#     for prop in properties['s.gas.g']:
#         curout[prop] = [prop in sgasgkeys]
#     for prop in properties['h.properties']:
#         curout[prop] = [prop in hpropkeys]
#     curout['sim.properties'] = [spropkeys]
#     curout['halo.propeties'] = [hpropkeys]
#     curout['sim.gas'] = [sgaskeys]
#     curout['sum.gas.g'] = [sgasgkeys]

#     df = pd.concat([df, curout], ignore_index=True)

# df.to_csv('/home/sackmann/Documents/MAP/akins/Justice_League_Code/Analysis/Data/cptmarvel_properties_check.csv', index=False)

pynbody.halo : Unable to load AHF substructure file; continuing without. To expose the underlying problem as an exception, pass ignore_missing_substructure=False to the AHFCatalogue constructor


Checking properties for snapshot 4096


In [5]:
z0haloids = [1, 2, 3, 5, 6, 7, 8, 10, 11, 13, 16]
sim = 'cptmarvel'
excel_path = 'Data/tracked_particles.xlsx'

with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    for z0haloid in z0haloids:
        key = f'{sim}_{z0haloid}'
        try:
            # Read the data from the HDF5 file
            df = pd.read_hdf('Data/tracked_particles.hdf5', key=key, stop=1000)
            # Write the DataFrame to a new sheet in the Excel file
            df.to_excel(writer, sheet_name=key, index=False)
        except KeyError:
            print(f"Key {key} not found in the HDF5 file.")

In [5]:
df = pd.read_hdf('Data/testing/tracked_marvel_particles.hdf5', key='cptmarvel_1')

# print(np.unique(df.time))

In [6]:
def read_tracked_particles(data, sim, haloid):
    '''
    -> Reads in gas particles tracked across a number of simulation satellites and calculates/appends desired particle 
        properties for analysis.
    '''
    #--------------------------------#
    
    print(f'Loading tracked particles for {sim}-{haloid}...')
    
    key = f'{sim}_{str(int(haloid))}'

    # importing tracked particles.
    # 'v2' revision contains all tracked particles for the 19 satellite halos selected for this study (see README):
    
    time = np.unique(data.time)
    dt = time[1:]-time[:-1]
    dt = np.append(dt[0], dt)
    dt = dt[np.unique(data.time, return_inverse=True)[1]]
    data['dt'] = dt
    
    
    print('Successfully loaded')

    r_gal_prev = 0
    r_gal = np.array([])
    for t in np.unique(data.time):
        d = data[data.time==t]
        r_gas = np.mean(d.sat_r_gas)
        r_half = np.mean(d.sat_r_half)
        print(f' -----{d.time.iloc[0]}-----\nr_half avg: {r_half}\n full r_half: {np.array(d.sat_r_half)}\n ------------------')
        print(f'r_gas avg: {r_gas}\n full r_gas: {np.array(d.sat_r_gas)}\n ------------------')
        rg = np.max([r_gas,r_half])

        if np.isnan(rg) :
            rg = r_gal_prev

        # print(f't = {t:1f} Gyr, satellite R_gal = {rg:.2f} kpc')
        r_gal = np.append(r_gal,[rg]*len(d))

        r_gal_prev = rg

    data['r_gal'] = r_gal
    
    # r_gal_prev = 0
    # r_gal = np.array([])
    # for t in np.unique(data.time):
    #     d = data[data.time==t]
    #     r_gas = np.mean(d.host_r_gas)
    #     r_half = np.mean(d.host_r_half)
    #     rg = np.max([r_gas,r_half])

    #     if np.isnan(rg):

    #         rg = r_gal_prev

    #     # print(f't = {t:1f} Gyr, host R_gal = {rg:.2f} kpc')
    #     r_gal = np.append(r_gal,[rg]*len(d))

    #     r_gal_prev = rg

    # data['host_r_gal'] = r_gal
    
    thermo_disk = (np.array(data.temp) < 1.2e4) & (np.array(data.rho) > 0.1)
    
    in_sat = np.array(data.in_sat)
    other_sat = np.array(data.in_other_sat)
    in_host = np.array(data.in_host) & ~in_sat & ~other_sat
    
    sat_disk = in_sat & thermo_disk
    sat_halo = in_sat & ~thermo_disk
    
    host_disk = in_host & thermo_disk
    host_halo = in_host & ~thermo_disk
    
    IGM = np.array(data.in_IGM)
    
    # basic location classifications.
    data['sat_disk'] = sat_disk
    data['sat_halo'] = sat_halo
    data['host_disk'] = host_disk
    data['host_halo'] = host_halo
    data['other_sat'] = other_sat
    data['IGM'] = IGM

    # # appending the virial mass of a particle's respective satellite.
    # timesteps = read_timesteps(sim)
    # ts = timesteps[timesteps.z0haloid==haloid]
    # ts = ts.rename({'mass':'sat_Mvir'}, axis=1)
    # ts = ts[['time','sat_Mvir']]
    # ts['sat_Mvir'] = ts['sat_Mvir'].astype('float')

    # data = pd.merge_asof(data, ts.sort_values('time'), left_on='time', right_on='time', direction='nearest', tolerance=1)

    print(f'> Returning <tracked_particle> dataset <')
    return data

In [17]:
data = read_tracked_particles(df, 'cptmarvel', 1)

Loading tracked particles for cptmarvel-1...
Successfully loaded
 -----0.4397579333952826-----
r_half avg: nan
 full r_half: [nan nan nan ... nan nan nan]
 ------------------
r_gas avg: 0.19508778117508258
 full r_gas: [0.19508778 0.19508778 0.19508778 ... 0.19508778 0.19508778 0.19508778]
 ------------------
 -----0.5000514038523343-----
r_half avg: nan
 full r_half: [nan nan nan ... nan nan nan]
 ------------------
r_gas avg: nan
 full r_gas: [nan nan nan ... nan nan nan]
 ------------------
 -----0.6775821779758461-----
r_half avg: nan
 full r_half: [nan nan nan ... nan nan nan]
 ------------------
r_gas avg: nan
 full r_gas: [nan nan nan ... nan nan nan]
 ------------------
 -----0.9857488047558165-----
r_half avg: 0.2746086741727043
 full r_half: [0.27460867 0.27460867 0.27460867 ... 0.27460867 0.27460867 0.27460867]
 ------------------
r_gas avg: 0.8053933876399381
 full r_gas: [0.80539339 0.80539339 0.80539339 ... 0.80539339 0.80539339 0.80539339]
 ------------------
 -----1.625

In [15]:
data.head(10)

,time,pid,rho,temp,mass,coolontime,r,r_per_Rvir,x,y,...,in_other_sat,in_IGM,dt,r_gal,sat_disk,sat_halo,host_disk,host_halo,other_sat,IGM
0,0.439758,4384223,0.000367,8200.709961,1406.783846,0.0,59.417271,47.836498,43.449372,-11.330487,...,False,True,0.060293,0.0,False,False,False,False,False,True
1,0.439758,4487567,0.000369,9834.842773,1406.783846,0.0,37.189712,29.941219,16.065771,-0.167573,...,False,True,0.060293,0.0,False,False,False,False,False,True
2,0.439758,4488463,0.000313,8158.950684,1406.783846,0.0,40.107012,32.289921,19.728380,-5.469099,...,False,True,0.060293,0.0,False,False,False,False,False,True
3,0.439758,4488543,0.000293,7853.397949,1406.783846,0.0,39.350841,31.681132,18.756004,-4.634972,...,False,True,0.060293,0.0,False,False,False,False,False,True
4,0.439758,4488559,0.000332,8223.921875,1406.783846,0.0,39.219601,31.575471,18.983170,-3.577226,...,False,True,0.060293,0.0,False,False,False,False,False,True
5,0.439758,4488569,0.000282,7644.250000,1406.783846,0.0,39.746107,31.999358,19.268106,-5.050869,...,False,True,0.060293,0.0,False,False,False,False,False,True
6,0.439758,4488573,0.000310,7823.794434,1406.783846,0.0,39.992621,32.197825,19.883350,-4.958164,...,False,True,0.060293,0.0,False,False,False,False,False,True
7,0.439758,4488575,0.000311,7755.563477,1406.783846,0.0,39.855029,32.087051,19.992589,-4.438207,...,False,True,0.060293,0.0,False,False,False,False,False,True
8,0.439758,4488590,0.000327,8068.634277,1406.783846,0.0,40.229633,32.388642,20.104193,-3.562853,...,False,True,0.060293,0.0,False,False,False,False,False,True
9,0.439758,4488591,0.000352,8196.463867,1406.783846,0.0,39.710790,31.970925,20.132735,-3.373796,...,False,True,0.060293,0.0,False,False,False,False,False,True


In [11]:
# for i in range(len(pids)):
#     pid = pids[i]

#     times = np.unique(df[df['in_sat'] == True].time)
#     print(f'{len(times)} times for unique particle: {pid}')

df_sat = df[df['in_sat'] == True]
df_halo = df[df['in_host'] == False]
df_other = df[df['in_other_sat'] == True]

print(f'Number of particles in satellite: {len(df_sat)}')
print(f'Number of particles in host halo: {len(df_halo)}')
print(f'Number of particles in other halos: {len(df_other)}')
print(f'Total number of particles: {len(df)}')
print(f'Checksum: {len(df_sat) + len(df_other)}')

Number of particles in satellite: 11870941
Number of particles in host halo: 34524495
Number of particles in other halos: 98883
Total number of particles: 46395436
Checksum: 11969824


43 times for unique particle: 4384223 ...

In [2]:
%load_ext autoreload
%autoreload 2
import os
import importlib
import pynbody
import numpy as np
from base import *
subdir = os.path.join(os.getcwd(),'ParticlePropertiesCalc')
if subdir not in sys.path:
    sys.path.append(subdir)
from ParticlePropertiesCalc.compiler import *
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [3]:
data = read_tracked_particles('cptmarvel', 1)

In [4]:
data.head(10)

,time,pid,rho,temp,mass,coolontime,r,r_per_Rvir,x,y,...,vz,v,r_half,r_gas,in_galaxy,in_IGM,dt,r_gal,gal_disk,gal_halo
0,13.731119,4384223,1.312777e-05,38053.515625,1406.783846,0.0,49.201708,0.912776,6.090797,25.007533,...,4.224495,9.074127,1.321048,0.091534,True,False,0.428754,1.321048,False,True
1,13.731119,4487567,4.184076e-06,15783.703125,1411.930233,0.0,85.751875,1.590845,-18.089632,-83.187191,...,-4.735709,26.845487,1.321048,0.091534,False,True,0.428754,1.321048,False,False
2,13.731119,4488463,7.448216e-07,8953.788086,1406.783846,0.0,241.956598,4.488711,127.861778,189.472953,...,-4.836866,16.741013,1.321048,0.091534,False,True,0.428754,1.321048,False,False
3,13.731119,4488543,2.998303e-07,5161.521484,1406.783846,0.0,286.315273,5.311640,101.026509,208.660899,...,-12.971349,20.571135,1.321048,0.091534,False,True,0.428754,1.321048,False,False
4,13.731119,4488559,1.070740e-07,3292.479492,1406.783846,0.0,346.521919,6.428577,176.399575,107.096726,...,-11.277989,12.020217,1.321048,0.091534,False,True,0.428754,1.321048,False,False
5,13.731119,4488569,5.757205e-07,8808.339844,1406.783846,0.0,238.927534,4.432516,88.455832,208.221780,...,-1.086393,3.118572,1.321048,0.091534,False,True,0.428754,1.321048,False,False
6,13.731119,4488573,1.706986e-06,14706.977539,1406.783846,0.0,209.982020,3.895527,114.438509,168.648394,...,-6.397264,23.317348,1.321048,0.091534,False,True,0.428754,1.321048,False,False
7,13.731119,4488575,2.170537e-07,5374.517578,1406.783846,0.0,300.346732,5.571948,97.275608,222.795023,...,-11.474603,18.879048,1.321048,0.091534,False,True,0.428754,1.321048,False,False
8,13.731119,4488590,3.037285e-07,5538.841309,1406.783846,0.0,256.672888,4.761723,81.668632,159.978526,...,-5.440586,7.578686,1.321048,0.091534,False,True,0.428754,1.321048,False,False
9,13.731119,4488591,1.434479e-07,5702.703613,1406.783846,0.0,409.474218,7.596450,148.810051,282.460110,...,-11.873251,13.430167,1.321048,0.091534,False,True,0.428754,1.321048,False,False


In [7]:
calc_ejected_expelled(data,'cptmarvel', 1)

Processing particles:   2%|▏         | 20251/1077872 [09:37<9:04:32, 32.37it/s] 